In [1]:


msft = yf.Ticker("MSFT")

msft_ohlc_df = msft.history(period="max", interval='1d', start='2019-10-02', end='2020-01-01')
msft_ohlc_df.index = pd.to_datetime(msft_ohlc_df.index, format='%Y-%M-%d')
msft_ohlc_df



,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-10-02,134.72,134.83,132.08,133.13,30521700,0.0,0
2019-10-03,133.43,135.21,131.72,134.75,24132900,0.0,0
2019-10-04,135.21,136.69,134.88,136.57,22897700,0.0,0
2019-10-07,135.60,136.62,135.48,135.58,15303700,0.0,0
2019-10-08,135.54,136.21,134.09,134.14,25550500,0.0,0
...,...,...,...,...,...,...,...
2019-12-24,156.24,156.47,155.88,156.14,8989200,0.0,0
2019-12-26,156.32,157.48,156.16,157.42,14520600,0.0,0
2019-12-27,158.19,158.29,156.97,157.71,18412800,0.0,0


# Initial loads

In [109]:
import yfinance as yf
import pandas as pd
from stockstats import StockDataFrame as sdf
import os

from bokeh.plotting import figure, ColumnDataSource, show, output_file
from bokeh.models.widgets import Dropdown
from bokeh.io import curdoc
from bokeh.layouts import column, gridplot

from bokeh.models import BooleanFilter, CDSView, Select, Range1d, HoverTool, Slider
from bokeh.palettes import Category20
from bokeh.models.formatters import NumeralTickFormatter



# Get df from yf 

In [10]:
#Pulls ticker df
def get_symbol_df(symbol=None,interval='1d'):
    try:
        tick = yf.Ticker(symbol)
        df = tick.history(period="max", interval=interval, start='2019-10-02', end='2020-01-01')
        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df["Date"],format='%Y-%M-%d')
        return df
    except: 
        print("Invalid Ticker")


## Constants for Bokeh Plot

In [29]:
class main_plot_constants:
    def __init__(self):
        self.W_PLOT = 800
        self.H_PLOT = 500
        self.TOOLS = 'pan,wheel_zoom,hover,reset'

# Main Stock price Plot

In [31]:
# plot basic stock prices
def plot_stock_price(ticker):
    #Calling constants
    c = main_plot_constants()
    
    #Get ticket data
    df = get_symbol_df(ticker)
    
    #Initiate CDS
    stock = ColumnDataSource(
    data=dict(Date=[], Open=[], Close=[], High=[], Low=[],index=[]))

    stock.data = stock.from_df(df)
    
    #Creating Plot frame
    p = figure(plot_width=c.W_PLOT, plot_height=c.H_PLOT, tools=c.TOOLS,
               title="Stock price", toolbar_location='above')

    inc = stock.data['Close'] > stock.data['Open']
    dec = stock.data['Open'] > stock.data['Close']
    view_inc = CDSView(source=stock, filters=[BooleanFilter(inc)])
    view_dec = CDSView(source=stock, filters=[BooleanFilter(dec)])

    # map dataframe indices to date strings and use as label overrides
    p.xaxis.major_label_overrides = {
        i+int(stock.data['index'][0]): date.strftime('%b %d') for i, date in enumerate(pd.to_datetime(stock.data["Date"]))
    }
    p.xaxis.bounds = (stock.data['index'][0], stock.data['index'][-1])
    
    
    p.segment(x0='index', x1='index', y0='Low', y1='High', color=RED, source=stock, view=view_inc)
    p.segment(x0='index', x1='index', y0='Low', y1='High', color=GREEN, source=stock, view=view_dec)

    p.vbar(x='index', width=VBAR_WIDTH, top='Open', bottom='Close', fill_color=BLUE, line_color=BLUE,
           source=stock,view=view_inc, name="price")
    p.vbar(x='index', width=VBAR_WIDTH, top='Open', bottom='Close', fill_color=RED, line_color=RED,
           source=stock,view=view_dec, name="price")

    p.legend.location = "top_left"
    p.legend.border_line_alpha = 0
    p.legend.background_fill_alpha = 0
    p.legend.click_policy = "mute"
    
    p.yaxis.formatter = NumeralTickFormatter(format='$ 0,0[.]000')
    
    # Add more ticks in the plot
    p.x_range.range_padding = 0.05
    p.xaxis.ticker.desired_num_ticks = 40
    p.xaxis.major_label_orientation = 3.14/4
    
    # Select specific tool for the plot
    price_hover = p.select(dict(type=HoverTool))

    # Choose, which glyphs are active by glyph name
    price_hover.names = ["price"]
    
    # Creating tooltips
    price_hover.tooltips = [("Datetime", "@Date"),
                            ("Open", "@Open{$0,0.00}"),
                            ("Close", "@Close{$0,0.00}"),
                           ("Volume", "@Volume{($ 0.00 a)}")]
    price_hover.formatters={"Date": 'datetime'}

    
    
    return p


# Define constants/configurations

VBAR_WIDTH = 0.2 
RED = Category20[7][6]
GREEN = Category20[5][4]

BLUE = Category20[3][0]
BLUE_LIGHT = Category20[3][1]

ORANGE = Category20[3][2]
PURPLE = Category20[9][8]
BROWN = Category20[11][10]



# RSI Plot

In [154]:
#RSI
def plot_rsi(stockdf):
    c = main_plot_constants()
    
    #CONSTANTS
    p_width = c.W_PLOT
    p_height = 100
    
    #wrapper of stockdf
    stock = sdf.retype(stockdf)
    cur_rsi = stock['rsi_14'].iloc[-1]
    cur_high = stock['high'].iloc[-1]

    p = figure(plot_width=p_width, plot_height=p_height, tools=TOOLS,
               title="RSI", toolbar_location='above',y_range=(0,100))
    
    #Instatiating source data
    source=ColumnDataSource(stock)
    
    #x-axis format
   
    
    #line_plot
    p.line(x='date',y='rsi_14',source=source)
    
    #Styllizing tickers
    p.xaxis.visible=False
    p.axis.minor_tick_line_color = None

    return p


# RSI Rough work

In [153]:
df = get_symbol_df(ticker)
stock = sdf.retype(df)
cur_rsi = stock['rsi_14'].iloc[-1]
cur_high = stock['high'].iloc[-1]

p = figure(plot_width=c.W_PLOT, plot_height=100, tools=TOOLS,
           title="RSI", toolbar_location='above',y_range=(0,100))

#Instatiating source data

source=ColumnDataSource(stock)

p.xaxis.major_label_overrides = {
    i+int(source.data['date'][0]): date.strftime('%b %d') for i, date in enumerate(pd.to_datetime(source.data["date"]))
}

p.line(x='date',y='rsi_14',source=source)
#Styllizing tickers
#p.xaxis.visible=True
p.axis.minor_tick_line_color = None
p.xaxis.visible=False
p.axis.minor_tick_line_color = None
p.yaxis.bounds=(0,100)

show(p)

# Main

In [155]:
#Initiate ticker
symbol = 'MSFT'
p_stock = plot_stock_price(symbol)

ticker = 'MSFT'
df = get_symbol_df(ticker)
rsi = plot_rsi(df)


grid = gridplot(
    children=[p_stock,rsi],
    #toolbar_location='right',
    sizing_mode='scale_width',
    plot_width=c.W_PLOT,
    ncols=1
)
show(grid)